In [1]:
!pip install langchain sentence-transformers faiss-cpu streamlit python-docx pdfplumber beautifulsoup4 requests


  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached langgraph-1.0.3-py3-none-any.whl.metadata (7.8 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached attrs-25.4.0-p

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

print("Setup successful!")

model = SentenceTransformer("all-MiniLM-L6-v2")

vec = model.encode(["hello world"])
print("Embedding shape:", vec.shape)

# FAISS test
dim = vec.shape[1]
index = faiss.IndexFlatIP(dim)
faiss.normalize_L2(vec)
index.add(vec)
print("Index size:", index.ntotal)


/opt/anaconda3/envs/new_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup successful!
Embedding shape: (1, 384)
Index size: 1


In [3]:
import os

folders = ["data_raw", "data_processed"]
for f in folders:
    os.makedirs(f, exist_ok=True)

print("Folders created:", folders)


Folders created: ['data_raw', 'data_processed']


In [4]:
import pdfplumber

def load_pdf(path):
    pages_text = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            pages_text.append(text)
    return pages_text



In [17]:
pages = load_pdf('data_raw/sample.pdf')
print(pages[:1])

['Somalia Flood Exposure Methodology Note\nAnalysis for 2024 HNRP\nThis technical note summarises the methodology used to calculate the number of people\npotentially exposed to flooding in Somalia in the 2024 Somalia Humanitarian Needs and\nResponse Plan (HNRP). The UN OCHA Centre for Humanitarian Data worked with awiderange\nof technical partners to develop a methodology that was then endorsed by the Somalia ICCG\nandHCT.\nDaily FloodScan (1998-2022) & WorldPop (2020 UN Adjusted) raster data wasanalysedtogain\nunderstanding of flood conditions across Somalia for both March-April-May (MAM) and\nOctober-November-December (OND) seasons. FloodScan daily flood fraction Standard Flood\nExposure Depiction (SFED) was aggregated to yearly seasonal maximum fraction composites\nforboththeMAMandONDseasonsforallyearsofhistoricalFloodScandata(1998-2022).\nTheyearlyseasonalSFEDcompositeswerethenprocessed/reclassifiedintwodistinctways:\n1. Compositesreclassifiedtobinaryusinga20percentfloodfractionthr

# pull text from any website URL so it can use that information for retrieval & answering questions.

In [20]:
import requests
from bs4 import BeautifulSoup

def load_url(url):
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # remove script + style
    for tag in soup(["script", "style"]):
        tag.decompose()
    
    text = soup.get_text(separator="\n")
    clean_text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])
    return clean_text

# Example:
text = load_url("https://example.com")
print(text[:500])


Example Domain
Example Domain
This domain is for use in documentation examples without needing permission. Avoid use in operations.
Learn more


In [21]:
def ingest_file(path_or_url):
    if path_or_url.startswith("http"):
        content = load_url(path_or_url)
        return [{"source": path_or_url, "text": content}]
    
    if path_or_url.endswith(".pdf"):
        pages = load_pdf(path_or_url)
        return [{"source": path_or_url, "text": p} for p in pages]
    
    if path_or_url.endswith(".csv"):
        rows = load_csv(path_or_url)
        return [{"source": path_or_url, "text": r} for r in rows]
    
    return []


In [22]:
test_pdf = "data_raw/sample.pdf"
docs = ingest_file(test_pdf)

print("Loaded:", len(docs), "items")
docs[:2]


Loaded: 2 items


[{'source': 'data_raw/sample.pdf',
  'text': 'Somalia Flood Exposure Methodology Note\nAnalysis for 2024 HNRP\nThis technical note summarises the methodology used to calculate the number of people\npotentially exposed to flooding in Somalia in the 2024 Somalia Humanitarian Needs and\nResponse Plan (HNRP). The UN OCHA Centre for Humanitarian Data worked with awiderange\nof technical partners to develop a methodology that was then endorsed by the Somalia ICCG\nandHCT.\nDaily FloodScan (1998-2022) & WorldPop (2020 UN Adjusted) raster data wasanalysedtogain\nunderstanding of flood conditions across Somalia for both March-April-May (MAM) and\nOctober-November-December (OND) seasons. FloodScan daily flood fraction Standard Flood\nExposure Depiction (SFED) was aggregated to yearly seasonal maximum fraction composites\nforboththeMAMandONDseasonsforallyearsofhistoricalFloodScandata(1998-2022).\nTheyearlyseasonalSFEDcompositeswerethenprocessed/reclassifiedintwodistinctways:\n1. Compositesreclass

# Processing & Chunking Text
Chunking is critical for RAG.
Instead of sending entire documents to the model, we break them into small meaningful pieces.

In [26]:
!pip install langchain
!pip install langchain-text-splitters


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [28]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

def chunk_documents(docs):
    """
    Input: docs = [{"source": "...", "text": "..."}]
    Output: list of chunks with metadata
    """
    processed = []

    for d in docs:
        chunks = text_splitter.split_text(d["text"])
        for i, c in enumerate(chunks):
            processed.append({
                "source": d["source"],
                "chunk_id": i,
                "text": c
            })

    return processed


In [29]:
docs = ingest_file("data_raw/sample.pdf")  # reuse from Step 2
chunks = chunk_documents(docs)

print("Original pages:", len(docs))
print("Total chunks:", len(chunks))
chunks[:2]  # show first two chunks


Original pages: 2
Total chunks: 5


[{'source': 'data_raw/sample.pdf',
  'chunk_id': 0,
  'text': 'Somalia Flood Exposure Methodology Note\nAnalysis for 2024 HNRP\nThis technical note summarises the methodology used to calculate the number of people\npotentially exposed to flooding in Somalia in the 2024 Somalia Humanitarian Needs and\nResponse Plan (HNRP). The UN OCHA Centre for Humanitarian Data worked with awiderange\nof technical partners to develop a methodology that was then endorsed by the Somalia ICCG\nandHCT.\nDaily FloodScan (1998-2022) & WorldPop (2020 UN Adjusted) raster data wasanalysedtogain\nunderstanding of flood conditions across Somalia for both March-April-May (MAM) and\nOctober-November-December (OND) seasons. FloodScan daily flood fraction Standard Flood\nExposure Depiction (SFED) was aggregated to yearly seasonal maximum fraction composites'},
 {'source': 'data_raw/sample.pdf',
  'chunk_id': 1,
  'text': 'Exposure Depiction (SFED) was aggregated to yearly seasonal maximum fraction composites\nforbot

In [30]:
import json

def save_chunks(chunks, filename="processed.json"):
    path = f"data_processed/{filename}"
    with open(path, "w") as f:
        json.dump(chunks, f, indent=2)
    print("Saved:", path)


In [31]:
save_chunks(chunks, "sample_chunks.json")


Saved: data_processed/sample_chunks.json


In [32]:
def load_chunks(filename="processed.json"):
    with open(f"data_processed/{filename}", "r") as f:
        return json.load(f)
